In [ ]:
# Colab: runtime -> GPU first
import torch, platform, os, sys, subprocess, json
print("Python:", sys.version)
print("CUDA available:", torch.cuda.is_available(), "| device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

!pip -q install matplotlib pandas scikit-learn tqdm opencv-python-headless torchvision


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
CUDA available: True | device: NVIDIA A100-SXM4-40GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, shutil, zipfile

# >>> CHANGE THIS if your zip has a different name or path
ZIP_GLOB = "/content/drive/MyDrive/D-Fire.zip"   # looks anywhere under MyDrive
matches = glob.glob(ZIP_GLOB, recursive=True)
assert matches, f"No zip found matching {ZIP_GLOB}"
ZIP_PATH = matches[0]
print("Using ZIP:", ZIP_PATH)

# Unzip to /content/DFire (fresh)
DST_ROOT = "/content/DFire"
if os.path.exists(DST_ROOT):
    shutil.rmtree(DST_ROOT)
os.makedirs(DST_ROOT, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH) as z:
    z.extractall(DST_ROOT)

print("Unzipped to:", DST_ROOT)
!ls -R /content/DFire | head -n 200


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using ZIP: /content/drive/MyDrive/D-Fire.zip
Unzipped to: /content/DFire
/content/DFire:
test
train

/content/DFire/test:
images
labels

/content/DFire/test/images:
AoF06723.jpg
AoF06724.jpg
AoF06725.jpg
AoF06726.jpg
AoF06727.jpg
AoF06728.jpg
AoF06729.jpg
AoF06730.jpg
AoF06731.jpg
AoF06732.jpg
AoF06733.jpg
AoF06734.jpg
AoF06735.jpg
AoF06736.jpg
AoF06737.jpg
AoF06738.jpg
AoF06739.jpg
AoF06740.jpg
AoF06741.jpg
AoF06742.jpg
AoF06743.jpg
AoF06744.jpg
AoF06745.jpg
AoF06746.jpg
AoF06747.jpg
AoF06748.jpg
AoF06749.jpg
AoF06750.jpg
AoF06751.jpg
AoF06752.jpg
AoF06753.jpg
AoF06754.jpg
AoF06755.jpg
AoF06756.jpg
AoF06757.jpg
AoF06758.jpg
AoF06759.jpg
AoF06760.jpg
AoF06761.jpg
AoF06762.jpg
AoF06763.jpg
AoF06764.jpg
AoF06765.jpg
AoF06766.jpg
AoF06767.jpg
AoF06768.jpg
AoF06769.jpg
AoF06770.jpg
AoF06771.jpg
AoF06772.jpg
AoF06773.jpg
AoF06774.jpg
AoF06775.jpg
AoF06776.jpg
AoF0

In [ ]:
import os, glob

def find_dfire_root(root="/content/DFire"):
    # Look for .../train/images and .../test/images
    train_imgs = glob.glob(os.path.join(root, "**/train/images"), recursive=True)
    test_imgs  = glob.glob(os.path.join(root, "**/test/images"),  recursive=True)
    cands = set(os.path.dirname(os.path.dirname(p)) for p in train_imgs) & set(os.path.dirname(os.path.dirname(p)) for p in test_imgs)
    assert cands, "Could not find train/images and test/images inside the unzipped folder."
    # Pick the shortest path as the likely root
    return sorted(cands, key=len)[0]

DATA_SRC = find_dfire_root("/content/DFire")
WORKDIR  = "/content/drive/MyDrive/dfire_colab"
os.makedirs(WORKDIR, exist_ok=True)

print("DATA_SRC =", DATA_SRC)
print("Expected structure:")
print(os.path.exists(os.path.join(DATA_SRC, "train", "images")), "train/images")
print(os.path.exists(os.path.join(DATA_SRC, "train", "labels")), "train/labels")
print(os.path.exists(os.path.join(DATA_SRC, "test", "images")),  "test/images")
print(os.path.exists(os.path.join(DATA_SRC, "test", "labels")),  "test/labels")



DATA_SRC = /content/DFire
Expected structure:
True train/images
True train/labels
True test/images
True test/labels


In [ ]:
import torch, sys
print("CUDA available:", torch.cuda.is_available(),
      "| device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
!pip -q install matplotlib pandas scikit-learn tqdm opencv-python-headless torchvision


CUDA available: True | device: NVIDIA A100-SXM4-40GB


In [ ]:
# Assumes:
# DATA_SRC points to folder with: train/images, train/labels, test/images, test/labels
# WORKDIR is where outputs will be saved (e.g., "/content/drive/MyDrive/dfire_colab")

import os, glob, random, shutil
from pathlib import Path
random.seed(42)

def collect_images(root):
    imgs=[]
    for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp"):
        imgs += glob.glob(os.path.join(root, "**", ext), recursive=True)
    return imgs

def yolo_positive_stems(lbl_dir):
    pos=set()
    for p in glob.glob(os.path.join(lbl_dir, "**/*.txt"), recursive=True):
        with open(p, "r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                if line.strip():
                    pos.add(Path(p).stem)
                    break
    return pos

def copy_to_class_folder(files, pos_stems, out_split_root):
    os.makedirs(os.path.join(out_split_root, "fire"), exist_ok=True)
    os.makedirs(os.path.join(out_split_root, "nonfire"), exist_ok=True)
    kept=0
    for im in files:
        stem = Path(im).stem
        cls = "fire" if stem in pos_stems else "nonfire"
        dst = os.path.join(out_split_root, cls, os.path.basename(im))
        if not os.path.exists(dst):
            shutil.copy2(im, dst); kept+=1
    return kept

SRC_TRAIN_IMG = os.path.join(DATA_SRC, "train", "images")
SRC_TRAIN_LBL = os.path.join(DATA_SRC, "train", "labels")
SRC_TEST_IMG  = os.path.join(DATA_SRC, "test", "images")
SRC_TEST_LBL  = os.path.join(DATA_SRC, "test", "labels")

OUT_ROOT = os.path.join(WORKDIR, "dfire_cls")
for s in ["train","val","test"]:
    for c in ["fire","nonfire"]:
        os.makedirs(os.path.join(OUT_ROOT, s, c), exist_ok=True)

train_imgs = collect_images(SRC_TRAIN_IMG)
random.shuffle(train_imgs)
n = len(train_imgs); n_val = max(1, int(0.15*n))
train_part, val_part = train_imgs[n_val:], train_imgs[:n_val]

train_pos = yolo_positive_stems(SRC_TRAIN_LBL)
_ = copy_to_class_folder(train_part, train_pos, os.path.join(OUT_ROOT, "train"))
_ = copy_to_class_folder(val_part,   train_pos, os.path.join(OUT_ROOT, "val"))

test_imgs = collect_images(SRC_TEST_IMG)
test_pos  = yolo_positive_stems(SRC_TEST_LBL)  # ok if empty
_ = copy_to_class_folder(test_imgs, test_pos, os.path.join(OUT_ROOT, "test"))

def count_dir(d): return sum(1 for _ in glob.iglob(os.path.join(d, "*")))
for s in ["train","val","test"]:
    for c in ["fire","nonfire"]:
        print(f"{s}/{c}:", count_dir(os.path.join(OUT_ROOT, s, c)))
print("ImageFolder ready at:", OUT_ROOT)


train/fire: 7985
train/nonfire: 6653
val/fire: 1403
val/nonfire: 1180
test/fire: 2301
test/nonfire: 2005
ImageFolder ready at: /content/drive/MyDrive/dfire_colab/dfire_cls


In [ ]:
# Installs and GPU check
import torch, sys, os, json, numpy as np, matplotlib.pyplot as plt
print("CUDA available:", torch.cuda.is_available(),
      "| device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
!pip -q install torchvision scikit-learn tqdm pandas opencv-python-headless

import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import (classification_report, confusion_matrix,
                             roc_auc_score, average_precision_score,
                             precision_recall_curve, roc_curve, auc)

DATA_ROOT = os.path.join(WORKDIR, "dfire_cls")
EXP_DIR   = os.path.join(WORKDIR, "exp_resnet18")
os.makedirs(EXP_DIR, exist_ok=True)

def get_loaders(root, img_size=224, batch=64, workers=2):
    mean=[0.485,0.456,0.406]; std=[0.229,0.224,0.225]
    tf_train = transforms.Compose([
        transforms.RandomResizedCrop(img_size, scale=(0.7,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.2,0.2,0.2,0.1),
        transforms.ToTensor(), transforms.Normalize(mean,std),
    ])
    tf_eval = transforms.Compose([
        transforms.Resize(int(img_size*1.15)),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(), transforms.Normalize(mean,std),
    ])
    tr = datasets.ImageFolder(os.path.join(root,"train"), tf_train)
    va = datasets.ImageFolder(os.path.join(root,"val"),   tf_eval)
    te = datasets.ImageFolder(os.path.join(root,"test"),  tf_eval)

    ys=[y for _,y in tr.samples]
    class_counts = np.bincount(ys, minlength=2)
    weights = torch.tensor(class_counts.sum()/np.maximum(class_counts,1), dtype=torch.float32)

    tr_loader = DataLoader(tr, batch_size=batch, shuffle=True,  num_workers=workers, pin_memory=True)
    va_loader = DataLoader(va, batch_size=batch, shuffle=False, num_workers=workers, pin_memory=True)
    te_loader = DataLoader(te, batch_size=batch, shuffle=False, num_workers=workers, pin_memory=True)
    return tr_loader, va_loader, te_loader, weights

device = "cuda" if torch.cuda.is_available() else "cpu"
tr, va, te, class_w = get_loaders(DATA_ROOT, img_size=224, batch=64, workers=2)

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_w.to(device))
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))

def evaluate(loader):
    model.eval()
    logits_list=[]; y_list=[]
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device)
            logits = model(x)
            logits_list.append(logits.cpu()); y_list.append(y)
    logits = torch.cat(logits_list); y = torch.cat(y_list).numpy()
    probs = logits.softmax(1)[:,1].numpy()
    preds = (probs>=0.5).astype(int)
    acc = (preds==y).mean()
    cm = confusion_matrix(y, preds, labels=[0,1])
    try:
        auroc = roc_auc_score(y, probs); aupr = average_precision_score(y, probs)
    except ValueError:
        auroc, aupr = float("nan"), float("nan")
    report = classification_report(y, preds, target_names=["nonfire","fire"], digits=4, zero_division=0)
    return {"acc":acc,"auroc":auroc,"aupr":aupr,"cm":cm,"y":y,"probs":probs,"report":report}

best, wait, patience = -1.0, 0, 5
for ep in range(1, 21):  # up to 20 epochs
    model.train(); running=0.0
    for x,y in tr:
        x,y=x.to(device), y.to(device)
        opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
        running += loss.item()*x.size(0)
    train_loss = running/len(tr.dataset)

    val = evaluate(va)
    score = (val["auroc"] if not np.isnan(val["auroc"]) else val["acc"])
    print(f"Epoch {ep:02d} | train_loss={train_loss:.4f} | val_acc={val['acc']:.4f} | val_auroc={val['auroc']:.4f}")
    torch.save(model.state_dict(), os.path.join(EXP_DIR,"last.pt"))
    if score > best:
        best=score; wait=0
        torch.save(model.state_dict(), os.path.join(EXP_DIR,"best.pt"))
        with open(os.path.join(EXP_DIR,"val_report.txt"),"w") as f: f.write(val["report"])
    else:
        wait+=1
        if wait>=patience:
            print("Early stopping."); break

# Test + artifacts
model.load_state_dict(torch.load(os.path.join(EXP_DIR,"best.pt"), map_location=device))
test = evaluate(te)

with open(os.path.join(EXP_DIR,"test_report.txt"),"w") as f:
    f.write(test["report"]); f.write(f"\nACC={test['acc']:.4f} AUROC={test['auroc']:.4f} AUPR={test['aupr']:.4f}\n")
with open(os.path.join(EXP_DIR,"test_metrics.json"),"w") as f:
    json.dump({k: (float(v) if isinstance(v,(int,float,np.floating)) else None)
               for k,v in test.items() if k in ["acc","auroc","aupr"]}, f, indent=2)

# Confusion matrix
plt.figure(figsize=(3.2,3.2))
plt.imshow(test["cm"])
for i in range(2):
    for j in range(2):
        plt.text(j,i,str(test["cm"][i,j]),ha="center",va="center")
plt.xticks([0,1],["nonfire","fire"]); plt.yticks([0,1],["nonfire","fire"])
plt.xlabel("Predicted"); plt.ylabel("True"); plt.tight_layout()
plt.savefig(os.path.join(EXP_DIR,"cm_test.png"), dpi=200); plt.close()

# PR & ROC
from sklearn.metrics import precision_recall_curve, roc_curve, auc, average_precision_score
p,r,_ = precision_recall_curve(test["y"], test["probs"])
ap = average_precision_score(test["y"], test["probs"])
plt.figure(); plt.step(r,p,where="post"); plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title(f"PR curve (AP={ap:.3f})"); plt.tight_layout(); plt.savefig(os.path.join(EXP_DIR,"curves_test_pr.png"), dpi=200); plt.close()

fpr,tpr,_ = roc_curve(test["y"], test["probs"])
aucv = auc(fpr,tpr)
plt.figure(); plt.plot(fpr,tpr); plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC curve (AUC={aucv:.3f})")
plt.tight_layout(); plt.savefig(os.path.join(EXP_DIR,"curves_test_roc.png"), dpi=200); plt.close()

print("Done. Artifacts saved to:", EXP_DIR)


CUDA available: True | device: NVIDIA A100-SXM4-40GB
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 228MB/s]
/tmp/ipython-input-3256477181.py:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))
/tmp/ipython-input-3256477181.py:81: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


Epoch 01 | train_loss=0.4346 | val_acc=0.7522 | val_auroc=0.8472


/tmp/ipython-input-3256477181.py:81: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


Epoch 02 | train_loss=0.2954 | val_acc=0.8107 | val_auroc=0.9003


...

Epoch 11 | train_loss=0.1020 | val_acc=0.8355 | val_auroc=0.9315


/tmp/ipython-input-3256477181.py:81: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


Epoch 12 | train_loss=0.0913 | val_acc=0.7998 | val_auroc=0.8959


/tmp/ipython-input-3256477181.py:81: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


Epoch 13 | train_loss=0.0860 | val_acc=0.8455 | val_auroc=0.9327
Early stopping.
Done. Artifacts saved to: /content/drive/MyDrive/dfire_colab/exp_resnet18


In [ ]:
# Uses WORKDIR/dfire_cls and WORKDIR/exp_resnet18 from earlier cells
import os, json, numpy as np, matplotlib.pyplot as plt
import torch, torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             average_precision_score, precision_recall_curve, roc_curve, auc)

DATA_ROOT = os.path.join(WORKDIR, "dfire_cls")
EXP_DIR   = os.path.join(WORKDIR, "exp_resnet18")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Eval transforms must match training eval pipeline (Resize->CenterCrop->Normalize for 224)
tf_eval = transforms.Compose([
    transforms.Resize(257),  # ~1.15 * 224
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

te_ds = datasets.ImageFolder(os.path.join(DATA_ROOT, "test"), tf_eval)
te_loader = DataLoader(te_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load(os.path.join(EXP_DIR, "best.pt"), map_location=device))
model.to(device).eval()

logits_list=[]; y_list=[]
with torch.no_grad():
    for x,y in te_loader:
        x = x.to(device)
        logits_list.append(model(x).cpu()); y_list.append(y)
logits = torch.cat(logits_list); y = torch.cat(y_list).numpy()
probs  = logits.softmax(1)[:,1].numpy()
preds  = (probs >= 0.5).astype(int)

acc = (preds==y).mean()
cm  = confusion_matrix(y, preds, labels=[0,1])
auroc = roc_auc_score(y, probs)
aupr  = average_precision_score(y, probs)
print(f"TEST  acc={acc:.4f}  AUROC={auroc:.4f}  AUPR={aupr:.4f}")

# Save reports & metrics
with open(os.path.join(EXP_DIR,"test_report.txt"),"w") as f:
    f.write(classification_report(y, preds, target_names=["nonfire","fire"], digits=4, zero_division=0))
    f.write(f"\nACC={acc:.4f} AUROC={auroc:.4f} AUPR={aupr:.4f}\n")
with open(os.path.join(EXP_DIR,"test_metrics.json"),"w") as f:
    json.dump({"acc":float(acc),"auroc":float(auroc),"aupr":float(aupr)}, f, indent=2)

# Confusion matrix
plt.figure(figsize=(3.2,3.2))
plt.imshow(cm, cmap="Blues")
for i in range(2):
    for j in range(2):
        plt.text(j,i,str(cm[i,j]),ha="center",va="center")
plt.xticks([0,1],["nonfire","fire"]); plt.yticks([0,1],["nonfire","fire"])
plt.xlabel("Predicted"); plt.ylabel("True"); plt.tight_layout()
plt.savefig(os.path.join(EXP_DIR,"cm_test.png"), dpi=200); plt.close()

# PR & ROC
p,r,_ = precision_recall_curve(y, probs)
ap = average_precision_score(y, probs)
plt.figure(); plt.step(r,p,where="post"); plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title(f"PR curve (AP={ap:.3f})"); plt.tight_layout(); plt.savefig(os.path.join(EXP_DIR,"curves_test_pr.png"), dpi=200); plt.close()

fpr,tpr,_ = roc_curve(y, probs)
aucv = auc(fpr,tpr)
plt.figure(); plt.plot(fpr,tpr); plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC curve (AUC={aucv:.3f})")
plt.tight_layout(); plt.savefig(os.path.join(EXP_DIR,"curves_test_roc.png"), dpi=200); plt.close()

print("Saved artifacts to:", EXP_DIR)


TEST  acc=0.8409  AUROC=0.9303  AUPR=0.9175
Saved artifacts to: /content/drive/MyDrive/dfire_colab/exp_resnet18


In [ ]:
!ls -lah /content/drive/MyDrive/dfire_colab/exp_resnet18


total 86M
-rw------- 1 root root 43M Sep  9 19:49 best.pt
-rw------- 1 root root 18K Sep  9 20:03 cm_test.png
-rw------- 1 root root 46K Sep  9 20:03 curves_test_pr.png
-rw------- 1 root root 55K Sep  9 20:03 curves_test_roc.png
-rw------- 1 root root 43M Sep  9 20:01 last.pt
-rw------- 1 root root  92 Sep  9 20:03 test_metrics.json
-rw------- 1 root root 363 Sep  9 20:03 test_report.txt
-rw------- 1 root root 326 Sep  9 19:49 val_report.txt


In [ ]:
# === ResNet-50 @ 384 on D-Fire ===
import os, json, numpy as np, matplotlib.pyplot as plt
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score

DATA_ROOT = os.path.join(WORKDIR, "dfire_cls")
EXP_DIR   = os.path.join(WORKDIR, "exp_resnet50_384")
IMG_SIZE  = 384
BATCH     = 80
EPOCHS    = 20
PATIENCE  = 5

os.makedirs(EXP_DIR, exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
use_cuda = (device == "cuda")

def get_loaders(root, img_size=384, batch=64, workers=2):
    mean=[0.485,0.456,0.406]; std=[0.229,0.224,0.225]
    tf_train = transforms.Compose([
        transforms.RandomResizedCrop(img_size, scale=(0.7,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.2,0.2,0.2,0.1),
        transforms.ToTensor(), transforms.Normalize(mean,std),
    ])
    tf_eval = transforms.Compose([
        transforms.Resize(int(img_size*1.15)),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(), transforms.Normalize(mean,std),
    ])
    tr = datasets.ImageFolder(os.path.join(root,"train"), tf_train)
    va = datasets.ImageFolder(os.path.join(root,"val"),   tf_eval)
    te = datasets.ImageFolder(os.path.join(root,"test"),  tf_eval)

    ys=[y for _,y in tr.samples]
    cc = np.bincount(ys, minlength=2)
    weights = torch.tensor(cc.sum()/np.maximum(cc,1), dtype=torch.float32)

    tr_loader = DataLoader(tr, batch_size=batch, shuffle=True,  num_workers=workers, pin_memory=True)
    va_loader = DataLoader(va, batch_size=batch, shuffle=False, num_workers=workers, pin_memory=True)
    te_loader = DataLoader(te, batch_size=batch, shuffle=False, num_workers=workers, pin_memory=True)
    return tr_loader, va_loader, te_loader, weights, tf_eval

tr, va, te, class_w, tf_eval = get_loaders(DATA_ROOT, IMG_SIZE, BATCH, workers=2)

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_w.to(device))
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
scaler = (torch.amp.GradScaler("cuda") if use_cuda else None)

def evaluate(loader):
    model.eval()
    logits_list=[]; y_list=[]
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device)
            logits = model(x)
            logits_list.append(logits.cpu()); y_list.append(y)
    logits = torch.cat(logits_list); y = torch.cat(y_list).numpy()
    probs = logits.softmax(1)[:,1].numpy()
    preds = (probs>=0.5).astype(int)
    acc = (preds==y).mean()
    try:
        auroc = roc_auc_score(y, probs); aupr = average_precision_score(y, probs)
    except ValueError:
        auroc, aupr = float("nan"), float("nan")
    return {"acc":acc,"auroc":auroc,"aupr":aupr}

best, wait = -1.0, 0
for ep in range(1, EPOCHS+1):
    model.train(); running=0.0
    for x,y in tr:
        x,y = x.to(device), y.to(device)
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast("cuda", enabled=use_cuda):
            logits = model(x)
            loss = criterion(logits, y)
        if scaler:
            scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
        else:
            loss.backward(); opt.step()
        running += loss.item()*x.size(0)
    tr_loss = running/len(tr.dataset)
    val = evaluate(va)
    score = (val["auroc"] if not np.isnan(val["auroc"]) else val["acc"])
    print(f"Epoch {ep:02d} | train_loss={tr_loss:.4f} | val_acc={val['acc']:.4f} | val_auroc={val['auroc']:.4f}")
    torch.save(model.state_dict(), os.path.join(EXP_DIR,"last.pt"))
    if score > best:
        best = score; wait = 0
        torch.save(model.state_dict(), os.path.join(EXP_DIR,"best.pt"))
        with open(os.path.join(EXP_DIR,"val_metrics.json"),"w") as f:
            json.dump(val, f, indent=2)
    else:
        wait += 1
        if wait >= PATIENCE:
            print("Early stopping."); break

# Test evaluation + plots
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc

def test_and_plot(exp_dir, img_size=IMG_SIZE):
    # reload with eval transform at the same size
    eval_tf = transforms.Compose([
        transforms.Resize(int(img_size*1.15)),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(), transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ])
    te_ds = datasets.ImageFolder(os.path.join(DATA_ROOT,"test"), eval_tf)
    te_loader = DataLoader(te_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

    m = models.resnet50(weights=None); m.fc = nn.Linear(m.fc.in_features, 2)
    m.load_state_dict(torch.load(os.path.join(exp_dir,"best.pt"), map_location=device))
    m = m.to(device).eval()

    logits_list=[]; y_list=[]
    with torch.no_grad():
        for x,y in te_loader:
            x = x.to(device)
            logits_list.append(m(x).cpu()); y_list.append(y)
    logits = torch.cat(logits_list); y = torch.cat(y_list).numpy()
    probs = logits.softmax(1)[:,1].numpy()
    preds = (probs>=0.5).astype(int)

    acc  = (preds==y).mean()
    auroc= roc_auc_score(y, probs)
    aupr = average_precision_score(y, probs)
    cm   = confusion_matrix(y, preds, labels=[0,1])
    print(f"TEST  acc={acc:.4f}  AUROC={auroc:.4f}  AUPR={aupr:.4f}")

    # save
    with open(os.path.join(exp_dir,"test_report.txt"),"w") as f:
        from sklearn.metrics import classification_report
        f.write(classification_report(y, preds, target_names=["nonfire","fire"], digits=4, zero_division=0))
        f.write(f"\nACC={acc:.4f} AUROC={auroc:.4f} AUPR={aupr:.4f}\n")
    with open(os.path.join(exp_dir,"test_metrics.json"),"w") as f:
        json.dump({"acc":float(acc),"auroc":float(auroc),"aupr":float(aupr)}, f, indent=2)

    # plots
    plt.figure(figsize=(3.2,3.2)); plt.imshow(cm, cmap="Blues")
    for i in range(2):
        for j in range(2):
            plt.text(j,i,str(cm[i,j]),ha="center",va="center")
    plt.xticks([0,1],["nonfire","fire"]); plt.yticks([0,1],["nonfire","fire"])
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.tight_layout()
    plt.savefig(os.path.join(exp_dir,"cm_test.png"), dpi=200); plt.close()

    p,r,_ = precision_recall_curve(y, probs)
    ap = average_precision_score(y, probs)
    plt.figure(); plt.step(r,p,where="post"); plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(f"PR curve (AP={ap:.3f})"); plt.tight_layout(); plt.savefig(os.path.join(exp_dir,"curves_test_pr.png"), dpi=200); plt.close()

    fpr,tpr,_ = roc_curve(y, probs)
    aucv = auc(fpr,tpr)
    plt.figure(); plt.plot(fpr,tpr); plt.plot([0,1],[0,1],'--')
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC curve (AUC={aucv:.3f})")
    plt.tight_layout(); plt.savefig(os.path.join(exp_dir,"curves_test_roc.png"), dpi=200); plt.close()

test_and_plot(EXP_DIR, IMG_SIZE)
print("Saved to:", EXP_DIR)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 221MB/s]


Epoch 01 | train_loss=0.3898 | val_acc=0.8014 | val_auroc=0.8743
Epoch 02 | train_loss=0.2433 | val_acc=0.8033 | val_auroc=0.8836
Epoch 03 | train_loss=0.1924 | val_acc=0.8386 | val_auroc=0.9115
Epoch 04 | train_loss=0.1605 | val_acc=0.7960 | val_auroc=0.9282
Epoch 05 | train_loss=0.1422 | val_acc=0.8277 | val_auroc=0.9164
Epoch 06 | train_loss=0.1249 | val_acc=0.7611 | val_auroc=0.8833
Epoch 07 | train_loss=0.1180 | val_acc=0.8451 | val_auroc=0.9265


KeyboardInterrupt: 

In [ ]:
# Evaluate exp_resnet50_384 on test
import os, json, numpy as np, matplotlib.pyplot as plt
import torch, torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score
from sklearn.metrics import precision_recall_curve, roc_curve, auc

EXP_DIR   = os.path.join(WORKDIR, "exp_resnet50_384")
DATA_ROOT = os.path.join(WORKDIR, "dfire_cls")
IMG_SIZE  = 384
device    = "cuda" if torch.cuda.is_available() else "cpu"

# test loader @ 384
tf_eval = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
te_ds = datasets.ImageFolder(os.path.join(DATA_ROOT, "test"), tf_eval)
te_loader = DataLoader(te_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

# load best checkpoint
model = models.resnet50(weights=None); model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load(os.path.join(EXP_DIR,"best.pt"), map_location=device))
model = model.to(device).eval()

# forward
logits_list=[]; y_list=[]
with torch.no_grad():
    for x,y in te_loader:
        x = x.to(device)
        logits_list.append(model(x).cpu()); y_list.append(y)
logits = torch.cat(logits_list); y = torch.cat(y_list).numpy()
probs  = logits.softmax(1)[:,1].numpy()
preds  = (probs >= 0.5).astype(int)

# metrics
acc  = (preds==y).mean()
auroc= roc_auc_score(y, probs)
aupr = average_precision_score(y, probs)
cm   = confusion_matrix(y, preds, labels=[0,1])
print(f"TEST  acc={acc:.4f}  AUROC={auroc:.4f}  AUPR={aupr:.4f}")

# save reports/plots
with open(os.path.join(EXP_DIR,"test_report.txt"),"w") as f:
    f.write(classification_report(y, preds, target_names=["nonfire","fire"], digits=4, zero_division=0))
    f.write(f"\nACC={acc:.4f} AUROC={auroc:.4f} AUPR={aupr:.4f}\n")
with open(os.path.join(EXP_DIR,"test_metrics.json"),"w") as f:
    json.dump({"acc":float(acc),"auroc":float(auroc),"aupr":float(aupr)}, f, indent=2)

plt.figure(figsize=(3.2,3.2)); plt.imshow(cm, cmap="Blues")
for i in range(2):
    for j in range(2):
        plt.text(j,i,str(cm[i,j]),ha="center",va="center")
plt.xticks([0,1],["nonfire","fire"]); plt.yticks([0,1],["nonfire","fire"])
plt.xlabel("Predicted"); plt.ylabel("True"); plt.tight_layout()
plt.savefig(os.path.join(EXP_DIR,"cm_test.png"), dpi=200); plt.close()

p,r,_ = precision_recall_curve(y, probs)
ap = average_precision_score(y, probs)
plt.figure(); plt.step(r,p,where="post"); plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title(f"PR curve (AP={ap:.3f})"); plt.tight_layout(); plt.savefig(os.path.join(EXP_DIR,"curves_test_pr.png"), dpi=200); plt.close()

fpr,tpr,_ = roc_curve(y, probs); aucv = auc(fpr,tpr)
plt.figure(); plt.plot(fpr,tpr); plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC curve (AUC={aucv:.3f})")
plt.tight_layout(); plt.savefig(os.path.join(EXP_DIR,"curves_test_roc.png"), dpi=200); plt.close()

print("Saved artifacts to:", EXP_DIR)


TEST  acc=0.7959  AUROC=0.9125  AUPR=0.8821
Saved artifacts to: /content/drive/MyDrive/dfire_colab/exp_resnet50_384


In [ ]:
# Choose threshold on VAL for ~90% recall, apply to TEST
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score
import numpy as np
import torch, torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# Assuming DATA_ROOT, EXP_DIR, IMG_SIZE, device, model are defined in previous cells

# build val loader @ 384
tf_eval = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
va_ds = datasets.ImageFolder(os.path.join(DATA_ROOT, "val"), tf_eval)
va_loader = DataLoader(va_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

# get val probs
val_probs=[]; val_y=[]
with torch.no_grad():
    for x,y_true in va_loader:
        x=x.to(device)
        val_probs.append(model(x).softmax(1)[:,1].cpu().numpy())
        val_y.append(y_true.numpy())

val_probs = np.concatenate(val_probs); val_y = np.concatenate(val_y)

p,r,t = precision_recall_curve(val_y, val_probs)
target_recall = 0.90
# Find the threshold that gives the closest recall to target_recall
if len(t) > 0:
  closest_recall_idx = np.argmin(np.abs(r - target_recall))
  # Ensure the index is within bounds of t
  thr = t[min(closest_recall_idx, len(t) - 1)]
else:
  thr = 0.5 # Default threshold if no thresholds are found

print("VAL threshold for ~90% recall:", float(thr))

# --- Re-calculate test probabilities and true labels ---
te_ds = datasets.ImageFolder(os.path.join(DATA_ROOT, "test"), tf_eval)
te_loader = DataLoader(te_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

test_probs = []; test_y = []
with torch.no_grad():
    for x, y_true in te_loader:
        x = x.to(device)
        test_probs.append(model(x).softmax(1)[:, 1].cpu().numpy())
        test_y.append(y_true.numpy())

test_probs = np.concatenate(test_probs)
test_y = np.concatenate(test_y)
# --- End of re-calculation ---

# apply threshold to test using the re-calculated test_y and test_probs
test_preds_thr = (test_probs >= thr).astype(int)

print("TEST @thr:",
      "precision=", precision_score(test_y, test_preds_thr, zero_division=0),
      "recall=",    recall_score(test_y, test_preds_thr,  zero_division=0),
      "f1=",        f1_score(test_y, test_preds_thr,      zero_division=0))

VAL threshold for ~90% recall: 0.1130559891462326
TEST @thr: precision= 0.782703172533681 recall= 0.8982543640897755 f1= 0.8365071992568509


In [ ]:
# Scan Drive for checkpoints
import os, glob, pandas as pd

hits = glob.glob("/content/drive/**/best.pt", recursive=True)
df = pd.DataFrame({"path": hits})
df["exp_dir"] = df["path"].apply(lambda p: os.path.dirname(p))
df["exp_name"] = df["exp_dir"].apply(os.path.basename)

# Heuristic guesses
def guess_arch(p):
    p = p.lower()
    return "resnet50" if "resnet50" in p else "resnet18"
def guess_img(p):
    p = p.lower()
    return 384 if "384" in p else 224

df["arch_guess"] = df["path"].apply(guess_arch)
df["img_guess"]  = df["path"].apply(guess_img)

# Show most relevant ones first (those likely synthetic: contains 'sd', 'syn', 'synfire')
priority = df["path"].str.contains(r"(sd|syn|synfire)", case=False, regex=True)
df = df.sort_values(["exp_name", "arch_guess"], ascending=True)
display(df[["exp_name","arch_guess","img_guess","path"]])
print("Found", len(df), "checkpoints.")


/tmp/ipython-input-1014606374.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  priority = df["path"].str.contains(r"(sd|syn|synfire)", case=False, regex=True)


,exp_name,arch_guess,img_guess,path
0,exp_resnet18,resnet18,224,/content/drive/MyDrive/dfire_colab/exp_resnet1...
1,exp_resnet50_384,resnet50,384,/content/drive/MyDrive/dfire_colab/exp_resnet5...


Found 2 checkpoints.
